In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/twitter-entity-sentiment-analysis/twitter_validation.csv
/kaggle/input/twitter-entity-sentiment-analysis/twitter_training.csv


In [27]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import tensorflow_hub as hub
import tensorflow_text as text
import tensorflow as tf

import spacy
import nltk

import warnings
warnings.filterwarnings('ignore')

In [28]:
data = pd.read_csv("/kaggle/input/twitter-entity-sentiment-analysis/twitter_training.csv", names=['id', 'Country', 'Label', 'Text'])
data.head()

,id,Country,Label,Text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


In [29]:
data.isna().sum()

id           0
Country      0
Label        0
Text       686
dtype: int64

In [30]:
data.dropna(inplace=True)

In [31]:
data.shape

(73996, 4)

In [32]:
data[data.duplicated()].shape

(2340, 4)

In [33]:
73996 - 2340

71656

In [34]:
data = data[~data.duplicated()]

In [35]:
data['Label'].value_counts()

Label
Negative      21698
Positive      19713
Neutral       17708
Irrelevant    12537
Name: count, dtype: int64

In [36]:
nlp = spacy.load('en_core_web_lg')

In [37]:
def preprocess(text):
    doc = nlp(text)
    filtered_tokens = []
    for token in doc:
        if token.is_stop or token.is_punct:
            continue
        filtered_tokens.append(token.lemma_)
    return ' '.join(filtered_tokens)

In [38]:
data['Preprocessed_text'] = data['Text'].apply(preprocess)

In [39]:
le = LabelEncoder()
data['Label_Labelled'] = le.fit_transform(data['Label'])

In [40]:
data.head(3)

,id,Country,Label,Text,Preprocessed_text,Label_Labelled
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...,m get borderland murder,3
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...,come border kill,3
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...,m get borderland kill,3


In [41]:
X = data['Preprocessed_text']
y = data['Label_Labelled']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2024, stratify=data['Label_Labelled'])

In [42]:
clf = Pipeline([
    ('vectorizer', TfidfVectorizer()), ('classifier', MultinomialNB())
])
clf.fit(X_train, y_train)

Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                ('classifier', MultinomialNB())])

In [43]:
print(classification_report(y_true=y_test, y_pred=clf.predict(X=X_test)))

              precision    recall  f1-score   support

           0       0.93      0.42      0.58      3761
           1       0.64      0.88      0.74      6510
           2       0.82      0.63      0.71      5312
           3       0.70      0.80      0.74      5914

    accuracy                           0.72     21497
   macro avg       0.77      0.68      0.69     21497
weighted avg       0.75      0.72      0.71     21497



In [44]:
clf = Pipeline([
    ('vectorizer', TfidfVectorizer()), ('classifier', RandomForestClassifier())
])
clf.fit(X_train, y_train)

Pipeline(steps=[('vectorizer', TfidfVectorizer()),
                ('classifier', RandomForestClassifier())])

In [45]:
print(classification_report(y_true=y_test, y_pred=clf.predict(X=X_test)))

              precision    recall  f1-score   support

           0       0.95      0.84      0.89      3761
           1       0.92      0.92      0.92      6510
           2       0.93      0.88      0.90      5312
           3       0.83      0.94      0.88      5914

    accuracy                           0.90     21497
   macro avg       0.91      0.89      0.90     21497
weighted avg       0.90      0.90      0.90     21497



In [46]:
X = data['Text']
y = data['Label_Labelled']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2024, stratify=data['Label_Labelled'])

In [48]:
!pip install transformers
!pip install torch
!pip install datasets

In [49]:
from transformers import BertTokenizer, BertForSequenceClassification

In [54]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

OSError: Can't load tokenizer for 'bert-base-uncased'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'bert-base-uncased' is the correct path to a directory containing all relevant files for a BertTokenizer tokenizer.

In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=4)

In [ ]:
def tokenize_function(example):
    return tokenizer(example['text'], padding='max_length', truncation=True)

# Assume you have a dataset in the form of 'datasets.Dataset'
tokenized_dataset = raw_dataset.map(tokenize_function, batched=True)